`tinynet.h5` and `valDataComplete.npz` files required to run this file.

To generate `tinynet.h5`, run `tiny-net.ipynb`

To generate `valDataComplete.npz` use `test-data-generate.ipynb`

In [40]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [41]:
# loading the test data
valData = np.load('valDataComplete.npz')
inputVal = valData['arr_0']
outputVal = valData['arr_1']

In [42]:
# standardizing/normalizing the data
augment = tf.keras.preprocessing.image.ImageDataGenerator(zca_whitening=True)
augment.mean = 112.69858580973307
augment.std = 70.93752549462194
augment.principal_components = 40

In [43]:
# tinynet architecture
def build(input_shape, num_classes):
    model = tf.keras.Sequential()
    
    # convolution layer 1
    model.add(tf.keras.layers.Convolution2D(64, (5, 5), strides=(1, 1), padding='same', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    # convolution layer 2
    model.add(tf.keras.layers.Convolution2D(128, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    # convolution layer 3
    model.add(tf.keras.layers.Convolution2D(128, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))

    # convolution layer 4
    model.add(tf.keras.layers.Convolution2D(256, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.ZeroPadding2D((1,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))

    # convolution layer 5
    model.add(tf.keras.layers.Convolution2D(256, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))

    # convolution layer 6
    model.add(tf.keras.layers.Convolution2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))

    # convolution layer 7
    model.add(tf.keras.layers.Convolution2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))

    # flattening
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.5))

    # fully connected layer 1
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu')) 
    model.add(tf.keras.layers.Dropout(0.5))
    
    # fully connected layer 2
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    # output layer
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    
    return model

In [44]:
# parameters
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.0005
batch_size = 256
input_shape = (64, 64, 3)
num_classes = 200

In [45]:
# initializing the model
model_tinynet = build(input_shape, num_classes)
model_tinynet.compile(optimizer=tfa.optimizers.weight_decay_optimizers.SGDW(
                        learning_rate=learning_rate, momentum=momentum, 
                        weight_decay=weight_decay, nesterov=True, name='SGDW'),
                        loss='categorical_crossentropy', 
                        metrics=['categorical_accuracy', 'top_k_categorical_accuracy'])

# loading the pretrained weights
model_tinynet.load_weights("tinynet.h5")

In [46]:
# running the model on test data
model_tinynet.evaluate(augment.flow(inputVal, outputVal))

  ...
    to  
  ['...']
313/313 [==============================] - 79s 253ms/step - loss: 2.9251 - categorical_accuracy: 0.4513 - top_k_categorical_accuracy: 0.7090


[2.925124935448741, 0.4513, 0.709]